# Assignment 4
## Description
In this assignment you must read in a file of metropolitan regions and associated sports teams from [assets/wikipedia_data.html](assets/wikipedia_data.html) and answer some questions about each metropolitan region. Each of these regions may have one or more teams from the "Big 4": NFL (football, in [assets/nfl.csv](assets/nfl.csv)), MLB (baseball, in [assets/mlb.csv](assets/mlb.csv)), NBA (basketball, in [assets/nba.csv](assets/nba.csv) or NHL (hockey, in [assets/nhl.csv](assets/nhl.csv)). Please keep in mind that all questions are from the perspective of the metropolitan region, and that this file is the "source of authority" for the location of a given sports team. Thus teams which are commonly known by a different area (e.g. "Oakland Raiders") need to be mapped into the metropolitan region given (e.g. San Francisco Bay Area). This will require some human data understanding outside of the data you've been given (e.g. you will have to hand-code some names, and might need to google to find out where teams are)!

For each sport I would like you to answer the question: **what is the win/loss ratio's correlation with the population of the city it is in?** Win/Loss ratio refers to the number of wins over the number of wins plus the number of losses. Remember that to calculate the correlation with [`pearsonr`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.pearsonr.html), so you are going to send in two ordered lists of values, the populations from the wikipedia_data.html file and the win/loss ratio for a given sport in the same order. Average the win/loss ratios for those cities which have multiple teams of a single sport. Each sport is worth an equal amount in this assignment (20%\*4=80%) of the grade for this assignment. You should only use data **from year 2018** for your analysis -- this is important!

## Notes

1. Do not include data about the MLS or CFL in any of the work you are doing, we're only interested in the Big 4 in this assignment.
2. I highly suggest that you first tackle the four correlation questions in order, as they are all similar and worth the majority of grades for this assignment. This is by design!
3. It's fair game to talk with peers about high level strategy as well as the relationship between metropolitan areas and sports teams. However, do not post code solving aspects of the assignment (including such as dictionaries mapping areas to teams, or regexes which will clean up names).
4. There may be more teams than the assert statements test, remember to collapse multiple teams in one city into a single value!

## Question 1
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NHL** using **2018** data.

In [ ]:
""""""
        # from IPython.core.display import HTML

        # def multi_table(table_list):
        #     ''' Acceps a list of IpyTable objects and returns a table which contains each IpyTable in a cell
        #     '''
        #     return HTML(
        #         '<table><tr style="background-color:white;">' + 
        #         ''.join(['<td>' + table._repr_html_() + '</td>' for table in table_list]) +
        #         '</tr></table>'
        #     )

        # multi_table([nhl_metro, metropol])
""""""

import pandas as pd
import numpy as np
import scipy.stats as stats
import re
pd.set_option('display.max_rows', None)

nhl_df=pd.read_csv("assets/nhl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nhl_correlation(): 
    # YOUR CODE HERE
    # pre-processing nhl_df
    nhl = nhl_df[nhl_df['year'] == 2018]
    num_cols = ['GP', 'W', 'L', 'OL', 'PTS', 'PTS%', 'GF', 'GA', 'SRS', 'SOS', 'RPt%', 'ROW', 'year']
    nhl_metro = nhl.drop(labels=[0,9,18,26])
    nhl_metro['team'] = nhl_metro.team.str.strip('*')
    nhl_metro[num_cols] =  nhl_metro[num_cols].astype(float)
    nhl_metro = nhl_metro[['team', 'W', 'L']]
    
    # preprocessing cities or metropolitian df
    metropol = cities.copy()
    metropol.drop(columns=['NFL','MLB','NBA'], inplace=True)
    metropol.rename(columns = {'Population (2016 est.)[8]':'Population', 'Metropolitan area':'Metro'}, inplace = True)
    metropol['Population'] =  metropol['Population'].astype(float)
    metropol['NHL'] = (metropol['NHL'].str.replace(r'\[.*', '')
                     .replace(r'', np.nan)
                     .replace(r'—', np.nan))
    metropol.dropna(inplace=True)
    metropol['NHL'] = metropol['NHL'].str.split(r'(?<=[a-z])(?=[A-Z])') # splitting 'team names' into a list
   
    # generating a new column of wl_ratios
    ratio = []
    for i, data in metropol.iterrows():
        pattern = '|'.join(data['NHL'])
        batch = nhl_metro[nhl_metro['team'].str.contains(pattern, na=False)]
        w_r = batch['W'].sum()/(batch['W'].sum() + batch['L'].sum())
        ratio.append(w_r)
    metropol['wl_ratio'] = ratio
    metropol = metropol[['Metro', 'Population', 'wl_ratio']].set_index('Metro')
    
    population_by_region = metropol['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = metropol['wl_ratio'].tolist() # pass in win/loss ratio from nhl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q1: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q1: There should be 28 teams being analysed for NHL"
    
    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()

nhl_correlation()

## Question 2
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NBA** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nba_df=pd.read_csv("assets/nba.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nba_correlation():
    # YOUR CODE HERE
    # preprocessing nba_df
    nba = nba_df[nba_df['year'] == 2018]
    nba['team'] = nba.team.str.replace('\*\s\(\d.*|\s\(\d.*', '')
    nba = nba[['team', 'W', 'L', 'W/L%']]
    nba[['W', 'L', 'W/L%']] = nba[['W', 'L', 'W/L%']].astype(float)
    
    # preprocessing cities or metropolitian df
    metropol = cities.copy()
    metropol.drop(columns=['NFL','MLB','NHL'], inplace=True)
    metropol.rename(columns = {'Population (2016 est.)[8]':'Population', 'Metropolitan area':'Metro'}, inplace = True)
    metropol['Population'] =  metropol['Population'].astype(float)
    metropol['NBA'] = (metropol['NBA'].str.replace(r'\[.*', '')
                         .replace(r'', np.nan)
                         .replace(r'—', np.nan))
    metropol.dropna(inplace=True)
    metropol['NBA'] = metropol['NBA'].str.split(r'(?<=[a-z])(?=[A-Z])')
    
    # generating wl_ratio column
    ratio = []
    for i, data in metropol.iterrows():
        pattern = '|'.join(data['NBA'])
        batch = nba[nba['team'].str.contains(pattern, na=False)]
        w_r = batch['W/L%'].sum()/len(batch)
        ratio.append(w_r)
    metropol['wl_ratio'] = ratio
    metropol = metropol[['Metro', 'Population', 'wl_ratio']].set_index('Metro')
    
    population_by_region = metropol['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = metropol['wl_ratio'].tolist() # pass in win/loss ratio from nba_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q2: Your lists must be the same length"
    assert len(population_by_region) == 28, "Q2: There should be 28 teams being analysed for NBA"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
    
nba_correlation()

## Question 3
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **MLB** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def mlb_correlation(): 
    # YOUR CODE HERE
    # preprocessing mlb_df
    mlb = mlb_df[mlb_df['year'] == 2018]
    mlb = mlb[['team', 'W', 'L', 'W-L%']]
    mlb[['W', 'L', 'W-L%']] = mlb[['W', 'L', 'W-L%']].astype(float)
    
    # preprocessing cities or metropolian df
    metropol = cities.copy()
    metropol.drop(columns=['NFL','NBA','NHL'], inplace=True)
    metropol.rename(columns = {'Population (2016 est.)[8]':'Population', 'Metropolitan area':'Metro'}, inplace = True)
    metropol['Population'] =  metropol['Population'].astype(float)
    metropol['MLB'] = (metropol['MLB'].str.replace(r'\[.*', '')
                             .replace(r'', np.nan)
                             .replace(r'—', np.nan))
    metropol.dropna(inplace=True)
    metropol['MLB'] = metropol['MLB'].str.split(r'(?<=[a-z])(?=[A-Z])')
    
    # generating wl_ratio
    ratio = []
    for i, data in metropol.iterrows():
        pattern = '|'.join(data['MLB'])
        batch = mlb[mlb['team'].str.contains(pattern, na=False)]
        w_r = batch['W-L%'].sum()/len(batch)
        ratio.append(w_r)
    metropol['wl_ratio'] = ratio
    metropol = metropol[['Metro', 'Population', 'wl_ratio']].set_index('Metro')
    
    population_by_region = metropol['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region = metropol['wl_ratio'].tolist() # pass in win/loss ratio from mlb_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q3: Your lists must be the same length"
    assert len(population_by_region) == 26, "Q3: There should be 26 teams being analysed for MLB"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
    
mlb_correlation()

## Question 4
For this question, calculate the win/loss ratio's correlation with the population of the city it is in for the **NFL** using **2018** data.

In [ ]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def nfl_correlation(): 
    # YOUR CODE HERE
    #
    nfl = nfl_df[nfl_df['year'] == 2018]
    nfl = nfl.drop(labels=[0,5,10,15,20,25,30,35])
    nfl = nfl[['team', 'W', 'L', 'W-L%']]
    nfl['team'] = nfl.team.str.strip('*|+')
    nfl[['W', 'L', 'W-L%']] = nfl[['W', 'L', 'W-L%']].astype(float)
    
    #
    metropol = cities.copy()
    metropol.drop(columns=['MLB','NBA','NHL'], inplace=True)
    metropol.rename(columns = {'Population (2016 est.)[8]':'Population', 'Metropolitan area':'Metro'}, inplace = True)
    metropol['Population'] =  metropol['Population'].astype(float)
    metropol['NFL'] = (metropol['NFL'].str.replace(r'\[.*|—\s\[.*', '')
                            .replace(r'', np.nan)
                            .replace(r'—', np.nan))
    metropol.dropna(inplace=True)
    metropol['NFL'] = metropol['NFL'].str.split(r'(?<=[a-z])(?=[A-Z])')
    
    #
    ratio = []
    for i, data in metropol.iterrows():
        pattern = '|'.join(data['NFL'])
        batch = nfl[nfl['team'].str.contains(pattern, na=False)]
        w_r = batch['W-L%'].sum()/len(batch)
        ratio.append(w_r)
    metropol['wl_ratio'] = ratio
    metropol = metropol[['Metro', 'Population', 'wl_ratio']].set_index('Metro')
    
    population_by_region =  metropol['Population'].tolist() # pass in metropolitan area population from cities
    win_loss_by_region =  metropol['wl_ratio'].tolist() # pass in win/loss ratio from nfl_df in the same order as cities["Metropolitan area"]

    assert len(population_by_region) == len(win_loss_by_region), "Q4: Your lists must be the same length"
    assert len(population_by_region) == 29, "Q4: There should be 29 teams being analysed for NFL"

    return stats.pearsonr(population_by_region, win_loss_by_region)[0]
    raise NotImplementedError()
    
nfl_correlation()

## Question 5
In this question I would like you to explore the hypothesis that **given that an area has two sports teams in different sports, those teams will perform the same within their respective sports**. How I would like to see this explored is with a series of paired t-tests (so use [`ttest_rel`](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.ttest_rel.html)) between all pairs of sports. Are there any sports where we can reject the null hypothesis? Again, average values where a sport has multiple teams in one region. Remember, you will only be including, for each sport, cities which have teams engaged in that sport, drop others as appropriate. This question is worth 20% of the grade for this assignment.

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as stats
import re

mlb_df=pd.read_csv("assets/mlb.csv")
nhl_df=pd.read_csv("assets/nhl.csv")
nba_df=pd.read_csv("assets/nba.csv")
nfl_df=pd.read_csv("assets/nfl.csv")
cities=pd.read_html("assets/wikipedia_data.html")[1]
cities=cities.iloc[:-1,[0,3,5,6,7,8]]

def fill_nanNsplit(df, col='NFL'):
    return (df[col].str.replace(r'\[.*|—\s\[.*', '')
                        .replace(r'', np.nan)
                        .replace(r'—', np.nan)
                        .str.split(r'(?<=[a-z])(?=[A-Z])'))

def gen_ratio(source,target, col='NFL'):
    ratio = []
    for i, data in target.iterrows():
        pattern = '|'.join(data[col])
        batch = source[source['team'].str.contains(pattern, na=False)]
        w_r = batch['W-L%'].sum()/len(batch)
        ratio.append(w_r)
    return ratio

def combinate(dfL, i=0):
    comb = []
    for x in np.array(np.meshgrid([i], [1,2,3,4])).T.reshape(-1, 2) :
        #print(x[0], x[1])
        M = pd.merge(dfL[x[0]], dfL[x[1]-1], how='inner', left_index=True, right_index=True)
        comb.append(stats.ttest_rel(M['wl_ratio_x'], M['wl_ratio_y'])[1])
    return comb


def sports_team_performance():
    # YOUR CODE HERE
    # metropolitian  or cities df 
    metropol = cities.copy()
    metropol.rename(columns = {'Population (2016 est.)[8]':'Population', 'Metropolitan area':'Metro'}, inplace = True)
    metropol['Population'] =  metropol['Population'].astype(float)
    metropol['NFL'] = fill_nanNsplit(metropol, col='NFL')
    metropol['NHL'] = fill_nanNsplit(metropol, col='NHL')
    metropol['NBA'] = fill_nanNsplit(metropol, col='NBA')
    metropol['MLB'] = fill_nanNsplit(metropol, col='MLB')
    # dropping unwanted columns
    metro_nfl = metropol[['Metro', 'Population', 'NFL']].dropna()
    metro_nhl = metropol[['Metro', 'Population', 'NHL']].dropna()
    metro_nba = metropol[['Metro', 'Population', 'NBA']].dropna()
    metro_mlb = metropol[['Metro', 'Population', 'MLB']].dropna()
    
    # reading nhl_df
    nhl = nhl_df[nhl_df['year'] == 2018]
    nhl = nhl.drop(labels=[0,9,18,26])
    nhl['team'] = nhl.team.str.strip('*|+')
    nhl = nhl[['team', 'W', 'L']]
    nhl[['W', 'L']] = nhl[['W', 'L']].astype(float)
    nhl['W-L%'] = nhl['W']/(nhl['W'] + nhl['L'])
    
    # reading nba_df
    nba = nba_df[nba_df['year'] == 2018]
    nba['team'] = nba.team.str.replace('\*\s\(\d.*|\s\(\d.*', '')
    nba = nba[['team', 'W', 'L', 'W/L%']]
    nba[['W', 'L', 'W-L%']] = nba[['W', 'L', 'W/L%']].astype(float)
    nba.drop(columns=['W/L%'], inplace=True)
    
    # reading nfl_df
    nfl = nfl_df[nfl_df['year'] == 2018]
    nfl = nfl.drop(labels=[0,5,10,15,20,25,30,35])
    nfl = nfl[['team', 'W', 'L', 'W-L%']]
    nfl['team'] = nfl.team.str.strip('*|+')
    nfl[['W', 'L', 'W-L%']] = nfl[['W', 'L', 'W-L%']].astype(float)
    
    # reading mlb_df
    mlb = mlb_df[mlb_df['year'] == 2018]
    mlb = mlb[['team', 'W', 'L', 'W-L%']]
    mlb[['W', 'L', 'W-L%']] = mlb[['W', 'L', 'W-L%']].astype(float)
    
    # generating w_l ratio for all dfs
    metro_nhl['wl_ratio'] = gen_ratio(nhl, metro_nhl, col='NHL')
    metro_nba['wl_ratio'] = gen_ratio(nba, metro_nba, col='NBA')
    metro_nfl['wl_ratio'] = gen_ratio(nfl, metro_nfl, col='NFL')
    metro_mlb['wl_ratio'] = gen_ratio(mlb, metro_mlb, col='MLB')

    metro_nhl = metro_nhl[['Metro', 'wl_ratio']].set_index('Metro')
    metro_nba = metro_nba[['Metro', 'wl_ratio']].set_index('Metro')
    metro_nfl = metro_nfl[['Metro', 'wl_ratio']].set_index('Metro')
    metro_mlb = metro_mlb[['Metro', 'wl_ratio']].set_index('Metro')
    
    # calculating 'ttest' 
    area_dfL = [metro_nfl, metro_nba, metro_nhl, metro_mlb]
    sports = ['NFL', 'NBA', 'NHL', 'MLB']
    # all possible combination's stats.ttest_rel inside list 
    COMB = {}
    for i, name in enumerate(sports):
        COMB[name] = combinate(area_dfL, i=i)
    
    # Note: p_values is a full dataframe, so df.loc["NFL","NBA"] should be the same as df.loc["NBA","NFL"] and
    # df.loc["NFL","NFL"] should return np.nan
    p_values = pd.DataFrame({k:COMB[k] for k in sports}, index=sports)
    
    assert abs(p_values.loc["NBA", "NHL"] - 0.02) <= 1e-2, "The NBA-NHL p-value should be around 0.02"
    assert abs(p_values.loc["MLB", "NFL"] - 0.80) <= 1e-2, "The MLB-NFL p-value should be around 0.80"
    return p_values
    raise NotImplementedError()
    
sports_team_performance()

,NFL,NBA,NHL,MLB
NFL,NaN,0.937509,0.030318,0.803459
NBA,0.937509,NaN,0.022386,0.949566
NHL,0.030318,0.022386,NaN,0.000703
MLB,0.803459,0.949566,0.000703,NaN
